# WGBS on Ronit's Samples

Ran with no limit

```
#!/bin/bash
## Job Name
#SBATCH --job-name=hw-bsnlP
## Allocation Definition
#SBATCH --account=coenv
#SBATCH --partition=coenv
## Nodes
#SBATCH --nodes=1
## Walltime (days-hours:minutes:seconds format)
#SBATCH --time=10-00:00:00
## Memory per node
#SBATCH --mem=100G
#SBATCH --mail-type=ALL
#SBATCH --mail-user=sr320@uw.edu
## Specify the working directory for this job
#SBATCH --chdir=/gscratch/scrubbed/sr320/021921-hw-bsnP
  
  
  
# Directories and programs
bismark_dir="/gscratch/srlab/programs/Bismark-0.21.0"
bowtie2_dir="/gscratch/srlab/programs/bowtie2-2.3.4.1-linux-x86_64/"
samtools="/gscratch/srlab/programs/samtools-1.9/samtools"
reads_dir="/gscratch/srlab/sr320/data/cg/"
genome_folder="/gscratch/srlab/sr320/data/Cgig-genome/Crassostrea_gigas.oyster_v9.dna_sm.toplevel/"
  
source /gscratch/srlab/programs/scripts/paths.sh
  
  
  
#${bismark_dir}/bismark_genome_preparation \
#--verbose \
#--parallel 28 \
#--path_to_aligner ${bowtie2_dir} \
#${genome_folder}
  
  
#/zr3644_11_R2.fastp-trim.20201206.fq.gz
  
find ${reads_dir}*_R1.fastp-trim.20201206.fq.gz \
| xargs basename -s _R1.fastp-trim.20201206.fq.gz | xargs -n 1 -P 6 -I{} ${bismark_dir}/bismark \
--path_to_bowtie ${bowtie2_dir} \
-genome ${genome_folder} \
-p 4 \
--non_directional \
-1 ${reads_dir}{}_R1.fastp-trim.20201206.fq.gz \
-2 ${reads_dir}{}_R2.fastp-trim.20201206.fq.gz \
  
  
  
find *.bam | \
xargs basename -s .bam | \
xargs -I{} ${bismark_dir}/deduplicate_bismark \
--bam \
--paired \
{}.bam
  
  
  
${bismark_dir}/bismark_methylation_extractor \
--bedGraph --counts --scaffolds \
--multicore 14 \
--buffer_size 75% \
*deduplicated.bam
  
  
  
# Bismark processing report
  
${bismark_dir}/bismark2report
  
#Bismark summary report
  
${bismark_dir}/bismark2summary
  
  
  
# Sort files for methylkit and IGV
  
find *deduplicated.bam | \
xargs basename -s .bam | \
xargs -I{} ${samtools} \
sort --threads 28 {}.bam \
-o {}.sorted.bam
  
# Index sorted files for IGV
# The "-@ 16" below specifies number of CPU threads to use.
  
find *.sorted.bam | \
xargs basename -s .sorted.bam | \
xargs -I{} ${samtools} \
index -@ 28 {}.sorted.bam
```

```
[sr320@mox1 021421-hw-bsn]$ head *report* | grep "Mapping efficiency:"
Mapping efficiency:	37.7% 
Mapping efficiency:	37.7% 
Mapping efficiency:	37.9% 
Mapping efficiency:	37.7% 
Mapping efficiency:	37.4% 
Mapping efficiency:	38.1% 
Mapping efficiency:	38.3% 
Mapping efficiency:	38.5% 
Mapping efficiency:	39.0% 
Mapping efficiency:	38.1% 
Mapping efficiency:	38.2% 
Mapping efficiency:	34.2% 
Mapping efficiency:	37.2% 
Mapping efficiency:	38.6% 
Mapping efficiency:	38.5% 
Mapping efficiency:	38.5% 
Mapping efficiency:	37.3% 
Mapping efficiency:	37.5% 
Mapping efficiency:	37.3% 
Mapping efficiency:	36.8% 
Mapping efficiency:	38.1% 
Mapping efficiency:	37.2% 
Mapping efficiency:	37.8% 
Mapping efficiency:	38.8%
```    

multiQC https://gannet.fish.washington.edu/seashell/bu-mox/scrubbed/021421-hw-bsn/multiqc_report.html

**And with the `-score_min L,0,-0.6`**

```
#!/bin/bash
## Job Name
#SBATCH --job-name=hw-bs
## Allocation Definition
#SBATCH --account=srlab
#SBATCH --partition=srlab
## Nodes
#SBATCH --nodes=1
## Walltime (days-hours:minutes:seconds format)
#SBATCH --time=20-00:00:00
## Memory per node
#SBATCH --mem=100G
#SBATCH --mail-type=ALL
#SBATCH --mail-user=sr320@uw.edu
## Specify the working directory for this job
#SBATCH --chdir=/gscratch/scrubbed/sr320/021321-hw-bs
 
 
 
# Directories and programs
bismark_dir="/gscratch/srlab/programs/Bismark-0.21.0"
bowtie2_dir="/gscratch/srlab/programs/bowtie2-2.3.4.1-linux-x86_64/"
samtools="/gscratch/srlab/programs/samtools-1.9/samtools"
reads_dir="/gscratch/srlab/sr320/data/cg/"
genome_folder="/gscratch/srlab/sr320/data/Cgig-genome/Crassostrea_gigas.oyster_v9.dna_sm.toplevel/"
 
source /gscratch/srlab/programs/scripts/paths.sh
 
 
 
#${bismark_dir}/bismark_genome_preparation \
#--verbose \
#--parallel 28 \
#--path_to_aligner ${bowtie2_dir} \
#${genome_folder}
 
 
#/zr3644_11_R2.fastp-trim.20201206.fq.gz
 
find ${reads_dir}*_R1.fastp-trim.20201206.fq.gz \
| xargs basename -s _R1.fastp-trim.20201206.fq.gz | xargs -I{} ${bismark_dir}/bismark \
--path_to_bowtie ${bowtie2_dir} \
-genome ${genome_folder} \
-p 4 \
-score_min L,0,-0.6 \
--non_directional \
-1 ${reads_dir}{}_R1.fastp-trim.20201206.fq.gz \
-2 ${reads_dir}{}_R2.fastp-trim.20201206.fq.gz \
 
 
 
find *.bam | \
xargs basename -s .bam | \
xargs -I{} ${bismark_dir}/deduplicate_bismark \
--bam \
--paired \
{}.bam
 
 
 
${bismark_dir}/bismark_methylation_extractor \
--bedGraph --counts --scaffolds \
--multicore 14 \
--buffer_size 75% \
*deduplicated.bam
 
 
 
# Bismark processing report
 
${bismark_dir}/bismark2report
 
#Bismark summary report
 
${bismark_dir}/bismark2summary
 
 
 
# Sort files for methylkit and IGV
 
find *deduplicated.bam | \
xargs basename -s .bam | \
xargs -I{} ${samtools} \
sort --threads 28 {}.bam \
-o {}.sorted.bam
 
# Index sorted files for IGV
# The "-@ 16" below specifies number of CPU threads to use.
 
find *.sorted.bam | \
xargs basename -s .sorted.bam | \
xargs -I{} ${samtools} \
index -@ 28 {}.sorted.bam
 
 
 
# 
# 
# find *deduplicated.bismark.cov.gz \
# | xargs basename -s _R1_001_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz \
# | xargs -I{} ${bismark_dir}/coverage2cytosine \
# --genome_folder ${genome_folder} \
# -o {} \
# --merge_CpG \
# --zero_based \
# {}_R1_001_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
# 
# 
# #creating bedgraphs post merge
# 
# for f in *merged_CpG_evidence.cov
# do
#   STEM=$(basename "${f}" .CpG_report.merged_CpG_evidence.cov)
#   cat "${f}" | awk -F $'\t' 'BEGIN {OFS = FS} {if ($5+$6 >= 10) {print $1, $2, $3, $4}}' \
#   > "${STEM}"_10x.bedgraph
# done
# 
# 
# 
# for f in *merged_CpG_evidence.cov
# do
#   STEM=$(basename "${f}" .CpG_report.merged_CpG_evidence.cov)
#   cat "${f}" | awk -F $'\t' 'BEGIN {OFS = FS} {if ($5+$6 >= 5) {print $1, $2, $3, $4}}' \
#   > "${STEM}"_5x.bedgraph
# done
# 
# 
# #creating tab files with raw count for glms
# 
# for f in *merged_CpG_evidence.cov
# do
#   STEM=$(basename "${f}" .CpG_report.merged_CpG_evidence.cov)
#   cat "${f}" | awk -F $'\t' 'BEGIN {OFS = FS} {if ($5+$6 >= 10) {print $1, $2, $3, $4, $5, $6}}' \
#   > "${STEM}"_10x.tab
# done
# 
# 
# for f in *merged_CpG_evidence.cov
# do
#   STEM=$(basename "${f}" .CpG_report.merged_CpG_evidence.cov)
#   cat "${f}" | awk -F $'\t' 'BEGIN {OFS = FS} {if ($5+$6 >= 5) {print $1, $2, $3, $4, $5, $6}}' \
#   > "${STEM}"_5x.tab
# done
```

```
[sr320@mox1 021321-hw-bs]$ head *report* | grep "Mapping efficiency:"
Mapping efficiency:	61.6% 
Mapping efficiency:	61.6% 
Mapping efficiency:	62.2% 
Mapping efficiency:	61.9% 
Mapping efficiency:	62.4% 
Mapping efficiency:	62.1% 
Mapping efficiency:	62.5% 
Mapping efficiency:	62.1% 
Mapping efficiency:	62.7% 
Mapping efficiency:	62.3% 
Mapping efficiency:	62.7% 
Mapping efficiency:	61.3% 
Mapping efficiency:	61.7% 
Mapping efficiency:	62.5% 
Mapping efficiency:	62.4% 
Mapping efficiency:	62.6% 
Mapping efficiency:	61.6% 
Mapping efficiency:	61.6% 
Mapping efficiency:	61.8% 
Mapping efficiency:	61.8% 
Mapping efficiency:	62.3% 
Mapping efficiency:	61.6% 
Mapping efficiency:	61.9% 
Mapping efficiency:	61.9% 
```    

MultiQC https://gannet.fish.washington.edu/seashell/bu-mox/scrubbed/021321-hw-bs/multiqc_report.html